<span style='color:red'> NOTE: You can only pass the lab, when you provide both code and markdown </span>

Use Code for your analysis
Use Markdown to document and elaborate on your findings, conclusions, assertions, etc.

# DS_ML_I_P1: Dataset creation from raw data 
Provided is a list of Excel-Files that stem from a radar measurement using an array of 15 Antennas and a frequency sweep. Another Excel sheet provides information on the type of object that should be detected and its orientation.

The overall task is to load the data into **a single dataframe**, add the **proper information on object**, **orientation** and **the name of the image that shows the object** (which is not provided here)


## 1. Load the data and check proper loading
Load all the data into a single dataframe so that
* The name of the file is a separat column
* Only the first five columns and all rows per Sheet tab should be integrated (15 Tabs in total, one per Antenna)
* Sheet tab name should be the major index in a multiindex column dataframe
* Tab column names should be the minor index
* After this dataframe has been created the object information, orientation and image name should be added as separate columns by integrating the information from the specific excel sheet.

In [78]:
import pandas as pd
import glob
import os
import numpy as np


In [79]:
pd.options.display.max_columns=1000
pd.options.display.max_rows=20


### 1.1 Load Measurement Data 
Measurement data from multiple excel files are loaded, each containing measurement data from 15 antennas. 

The final dataframe is structured with antennas name as the major/1stlevel index column and the five measurement values as the 2nd level index column. 

A separate column `filename` is added for each row for getting the additional information later.


In [80]:
%%time
file_paths = glob.glob("P1b/Measurements_8_April_2023_IMP-SIMO/*.xls")

dfs =[]

for file in file_paths:
    file_data = []
    sheets = pd.read_excel(file, sheet_name=None, usecols=[0,1,2,3,4])

    for sheet_name, df_temp in sheets.items():
        multi_columns = [
            np.repeat(sheet_name,len(df_temp.columns)),
            df_temp.columns.to_list()
        ]
        # df_temp.columns = pd.MultiIndex.from_arrays(multi_columns, names=['major','minor'])
        df_temp.columns = pd.MultiIndex.from_arrays(multi_columns)

        file_data.append(df_temp)

    df_temp["filename"] = os.path.splitext(os.path.basename(file))[0]
    dfs.append(pd.concat(file_data, axis=1))

concatenated_data = pd.concat(dfs, ignore_index=True)

CPU times: total: 7.03 s
Wall time: 7.25 s


In [81]:
df_m = concatenated_data

### 1.2 Load Protocol Data
The measurement protocol excel files are loaded into one dataframe and each column name is translated accordingly to english

In [82]:
df_p = pd.read_excel("P1b/Messprotokoll_18_04_2023_open_V1.xlsx", skiprows=6, usecols="C:H").rename(columns={
    "Messung": "measurement", 
    "Gegenstand": "object", 
    "Postion": "position", 
    "Dateienname ": "filename", 
    "Bild ": "image",
    "Anmerkungen": "comments"
})

Set filename as dataframe's index for the later merge operation

In [83]:
df_p['filename'] = df_p['filename'].astype(str).str.strip()
df_p = df_p.set_index('filename')

### 1.3 Add Additional Information to Measurement Dataframe
The measurement dataframe is enriched with the additional information like the object information, position and image name as separate columns, by mapping the column filename to the relevant information in the protocol dataframe.

In [84]:
df_m['filename'] = df_m['filename'].astype(str).str.strip()

df_m['object'] = df_m['filename'].map(df_p['object'])
df_m['position'] = df_m['filename'].map(df_p['position'])
df_m['image'] = df_m['filename'].map(df_p['image'])

In [85]:
df = df_m

## 2. Print some statistics and analyze

### 2.1 Statistic Analysis Result
* A total of 3.000 rows across 79 columns were loaded. There are 46 columns with the data type `float64`, 30 columns with `int64`, and 3 columns with `object` type.

* The `image` column was read as `float64` instead of `object`. A transformation is likely needed to correctly represent the data later.
* The column object, position, and image containts missing values


In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 79 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   (ANT 1, DAC Value)       3000 non-null   float64
 1   (ANT 1, DAC Value RAW)   3000 non-null   int64  
 2   (ANT 1, Magnitute)       3000 non-null   float64
 3   (ANT 1, Phase)           3000 non-null   float64
 4   (ANT 1, Frequency)       3000 non-null   int64  
 5   (ANT 2, DAC Value)       3000 non-null   float64
 6   (ANT 2, DAC Value RAW)   3000 non-null   int64  
 7   (ANT 2, Magnitute)       3000 non-null   float64
 8   (ANT 2, Phase)           3000 non-null   float64
 9   (ANT 2, Frequency)       3000 non-null   int64  
 10  (ANT 3, DAC Value)       3000 non-null   float64
 11  (ANT 3, DAC Value RAW)   3000 non-null   int64  
 12  (ANT 3, Magnitute)       3000 non-null   float64
 13  (ANT 3, Phase)           3000 non-null   float64
 14  (ANT 3, Frequency)      

There are in total 420 rows with missing values

In [87]:
rows_with_missing_values = df[df.isna().any(axis=1)]
print(f"{len(rows_with_missing_values)} has missing values")

420 has missing values


### What does those missing values mean? What does it says about the data quality?

The following show the summary statistic of the final dataframe.

It was found from the description that the `Frequency` of all antennas are all equals in every rows and every columns with the frequency 2450000000.

In [88]:
df.describe()

ANT 1                                                        \
         DAC Value DAC Value RAW    Magnitute        Phase     Frequency   
count  3000.000000   3000.000000  3000.000000  3000.000000  3.000000e+03   
mean     -0.337100   1390.000000   -29.440857   -11.037441  2.450000e+09   
std       0.727482    346.419787    21.308478   100.516179  0.000000e+00   
min      -1.576100    800.000000   -76.132400  -179.746000  2.450000e+09   
25%      -0.956600   1095.000000   -49.023000   -98.424450  2.450000e+09   
50%      -0.337100   1390.000000   -34.079850   -24.726700  2.450000e+09   
75%       0.282400   1685.000000    -5.197550    71.956325  2.450000e+09   
max       0.901900   1980.000000     2.967040   179.945000  2.450000e+09   

             ANT 2                                                        \
         DAC Value DAC Value RAW    Magnitute        Phase     Frequency   
count  3000.000000   3000.000000  3000.000000  3000.000000  3.000000e+03   
mean     -0.337100   1390.000000   -28.584241    -8.454822  2.450000e+09   
std       0.727482    346.419787    20.660701   101.662437  0.000000e+00   
min      -1.576100    800.000000   -78.798000  -179.855000  2.450000e+09   
25%      -0.956600   1095.000000   -47.094225  -104.927500  2.450000e+09   
50%      -0.337100   1390.000000   -33.717600    -0.022706  2.450000e+09   
75%       0.282400   1685.000000    -5.459223    80.736000  2.450000e+09   
max       0.901900   1980.000000     2.600070   179.882000  2.450000e+09   

             ANT 3                                                        \
         DAC Value DAC Value RAW    Magnitute        Phase     Frequency   
count  3000.000000   3000.000000  3000.000000  3000.000000  3.000000e+03   
mean     -0.337100   1390.000000   -28.496339     6.162484  2.450000e+09   
std       0.727482    346.419787    19.896935   105.454240  0.000000e+00   
min      -1.576100    800.000000   -65.850600  -179.841000  2.450000e+09   
25%      -0.956600   1095.000000   -46.240275   -93.691275  2.450000e+09   
50%      -0.337100   1390.000000   -33.768350    20.151350  2.450000e+09   
75%       0.282400   1685.000000    -6.185160    95.470150  2.450000e+09   
max       0.901900   1980.000000     1.951980   179.968000  2.450000e+09   

             ANT 4                                                        \
         DAC Value DAC Value RAW    Magnitute        Phase     Frequency   
count  3000.000000   3000.000000  3000.000000  3000.000000  3.000000e+03   
mean     -0.337100   1390.000000   -27.625137    -5.295042  2.450000e+09   
std       0.727482    346.419787    20.533735   102.950631  0.000000e+00   
min      -1.576100    800.000000   -66.213900  -179.830000  2.450000e+09   
25%      -0.956600   1095.000000   -47.068525  -108.048500  2.450000e+09   
50%      -0.337100   1390.000000   -32.086750     0.879465  2.450000e+09   
75%       0.282400   1685.000000    -4.311635    87.005050  2.450000e+09   
max       0.901900   1980.000000     3.010070   179.962000  2.450000e+09   

             ANT 5                                                        \
         DAC Value DAC Value RAW    Magnitute        Phase     Frequency   
count  3000.000000   3000.000000  3000.000000  3000.000000  3.000000e+03   
mean     -0.337100   1390.000000   -28.421452   -14.991958  2.450000e+09   
std       0.727482    346.419787    20.865768   101.811997  0.000000e+00   
min      -1.576100    800.000000   -72.225700  -179.865000  2.450000e+09   
25%      -0.956600   1095.000000   -48.134000  -103.428500  2.450000e+09   
50%      -0.337100   1390.000000   -31.816800   -35.516350  2.450000e+09   
75%       0.282400   1685.000000    -4.999288    70.339400  2.450000e+09   
max       0.901900   1980.000000     2.312730   179.475000  2.450000e+09   

             ANT 6                                                        \
         DAC Value DAC Value RAW    Magnitute        Phase     Frequency   
count  3000.000000   3000.000000  3000.000000  3000.0000

This is then later checked and confirmed below, that every value in every `Frequency` column is constant, 2450000000. More details will be shown in the visualization

In [89]:
freq_columns = df.xs('Frequency', axis=1, level=1)
(freq_columns == 2450000000).all(axis=None)

np.True_

In [90]:
print(df['image'].dtype)
print(df['image'].unique())
print(df['image'].min())
print(df['image'].max())

float64
[nan  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43.]
1.0
43.0


## 3. Visualize the data
* Scatter Plot
* Box Plot
* Histogram

### 3.1 Flattening the dataframe
To make data visualization easier, the DataFrame is flattened. Instead of using multilevel columns, there is now a separate column specifying which antenna the value was taken from.

In [91]:
antenna_df = df.loc[:,df.columns.get_level_values(0).str.startswith('ANT')]
antenna_df = antenna_df.stack(level=0, future_stack=True).reset_index()
antenna_df.rename(columns={'level_0':'Index', 'level_1':'Antenna'}, inplace=True)
antenna_df

,Index,Antenna,DAC Value,DAC Value RAW,Magnitute,Phase,Frequency
0,0,ANT 1,-1.5761,800,-51.029200,-32.22020,2450000000
1,0,ANT 2,-1.5761,800,-49.357300,-36.46740,2450000000
2,0,ANT 3,-1.5761,800,-48.255700,-29.19570,2450000000
3,0,ANT 4,-1.5761,800,-49.068000,-34.92130,2450000000
4,0,ANT 5,-1.5761,800,-49.537400,-32.57220,2450000000
...,...,...,...,...,...,...,...
44995,2999,ANT 11,0.9019,1980,0.001814,1.21299,2450000000
44996,2999,ANT 12,0.9019,1980,1.910730,53.37160,2450000000
44997,2999,ANT 13,0.9019,1980,0.064425,83.02310,2450000000
44998,2999,ANT 14,0.9019,1980,1.615870,52.62040,2450000000


In [92]:
object_details_df = df.loc[:,~df.columns.get_level_values(0).str.startswith('ANT')]
object_details_df.columns = object_details_df.columns.droplevel(level=1)
object_details_df

,filename,object,position,image
0,1804202300,Ohne Kamera,NaN,NaN
1,1804202300,Ohne Kamera,NaN,NaN
2,1804202300,Ohne Kamera,NaN,NaN
3,1804202300,Ohne Kamera,NaN,NaN
4,1804202300,Ohne Kamera,NaN,NaN
...,...,...,...,...
2995,1804202349,NaN,NaN,NaN
2996,1804202349,NaN,NaN,NaN
2997,1804202349,NaN,NaN,NaN
2998,1804202349,NaN,NaN,NaN


In [93]:
final_df = antenna_df.merge(object_details_df, how='left', left_on='Index', right_index=True)
final_df

,Index,Antenna,DAC Value,DAC Value RAW,Magnitute,Phase,Frequency,filename,object,position,image
0,0,ANT 1,-1.5761,800,-51.029200,-32.22020,2450000000,1804202300,Ohne Kamera,NaN,NaN
1,0,ANT 2,-1.5761,800,-49.357300,-36.46740,2450000000,1804202300,Ohne Kamera,NaN,NaN
2,0,ANT 3,-1.5761,800,-48.255700,-29.19570,2450000000,1804202300,Ohne Kamera,NaN,NaN
3,0,ANT 4,-1.5761,800,-49.068000,-34.92130,2450000000,1804202300,Ohne Kamera,NaN,NaN
4,0,ANT 5,-1.5761,800,-49.537400,-32.57220,2450000000,1804202300,Ohne Kamera,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
44995,2999,ANT 11,0.9019,1980,0.001814,1.21299,2450000000,1804202349,NaN,NaN,NaN
44996,2999,ANT 12,0.9019,1980,1.910730,53.37160,2450000000,1804202349,NaN,NaN,NaN
44997,2999,ANT 13,0.9019,1980,0.064425,83.02310,2450000000,1804202349,NaN,NaN,NaN
44998,2999,ANT 14,0.9019,1980,1.615870,52.62040,2450000000,1804202349,NaN,NaN,NaN


## 4. Conclusion


### 📂 General Dataset Overview
 Identify and describe data sources (e.g., databases, CSVs, APIs).

 Document the number of records (rows) and features (columns).

 Determine the data types (numeric, categorical, datetime, etc.).

 Check units of measurement (if applicable).

 Record metadata (e.g., column names, descriptions, units).

### 📊 Feature-Level Summary
 Provide summary statistics:

Numeric: count, mean, median, min, max, standard deviation, percentiles.

Categorical: count of unique values, frequency of each category.

 Identify primary keys or identifiers (if present).

 Highlight target variable(s) for prediction or analysis.

 Note data ranges and value distributions.

 Detect constant or quasi-constant features (little to no variation).

### 🧹 Data Quality Overview
 Report missing values (count and percentage per column).

 Detect duplicates (rows or IDs).

 Check for data type mismatches (e.g., numbers stored as strings).

 Flag outliers or unexpected values.

### 📈 Visualization (Optional but Recommended)
 Histograms or boxplots for numerical data.

 Bar charts for categorical data.

 Heatmaps or pairplots to see relationships (if already relevant).

### 🗒️ Documentation
 Maintain a data dictionary (column descriptions, formats, value ranges).

 Note any initial assumptions or findings relevant to data quality or usage.

